# Database connectivity 

in Python refers to the ability of Python applications to communicate and interact with databases. This is usually achieved using modules or libraries that provide a consistent interface between the Python application and the database system.

In [1]:
import sqlite3

# Step 1: Connect to a database named 'sample.db'. If it doesn't exist, it will be created.
conn = sqlite3.connect('sample.db')

# Step 2: Create a table called `users` with an `id` and a `name`.
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
)
''')

# Step 3: Populate the table with some data.
users_data = [
    ('Alice',),
    ('Bob',),
    ('Charlie',),
]
cursor.executemany('INSERT INTO users (name) VALUES (?)', users_data)
conn.commit()

# Step 4: Query the table and retrieve all rows.
cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()

# Step 5: Close the connection.
conn.close()

rows


[(1, 'Alice'), (2, 'Bob'), (3, 'Charlie')]

Demo of CRUD (Create, Read, Update, Delete) operations

In [2]:
import sqlite3

def connect_to_db(db_path):
    return sqlite3.connect(db_path)

def create_user(conn, name):
    cursor = conn.cursor()
    cursor.execute("INSERT INTO users (name) VALUES (?)", (name,))
    conn.commit()
    return cursor.lastrowid

def read_users(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users")
    return cursor.fetchall()

def update_user(conn, user_id, new_name):
    cursor = conn.cursor()
    cursor.execute("UPDATE users SET name = ? WHERE id = ?", (new_name, user_id))
    conn.commit()

def delete_user(conn, user_id):
    cursor = conn.cursor()
    cursor.execute("DELETE FROM users WHERE id = ?", (user_id,))
    conn.commit()


In [3]:
db_path = 'sample.db'
    
# Connect to the database
conn = connect_to_db(db_path)
    
# Create a new user
new_user_id = create_user(conn, "Daisy")
print(f"User 'Daisy' added with ID {new_user_id}")
    
# Read all users
print("Users in the database:")
for user in read_users(conn):
    print(user)
    
# Update a user's name
update_user(conn, new_user_id, "Daisy Duck")
print(f"\nUpdated user {new_user_id}'s name to 'Daisy Duck'")
    
# Read all users after update
print("Users in the database after update:")
for user in read_users(conn):
    print(user)
    
# Delete a user
delete_user(conn, new_user_id)
print(f"\nDeleted user with ID {new_user_id}")
    
# Read all users after deletion
print("Users in the database after deletion:")
for user in read_users(conn):
    print(user)
    
# Close the connection
conn.close()


User 'Daisy' added with ID 4
Users in the database:
(1, 'Alice')
(2, 'Bob')
(3, 'Charlie')
(4, 'Daisy')

Updated user 4's name to 'Daisy Duck'
Users in the database after update:
(1, 'Alice')
(2, 'Bob')
(3, 'Charlie')
(4, 'Daisy Duck')

Deleted user with ID 4
Users in the database after deletion:
(1, 'Alice')
(2, 'Bob')
(3, 'Charlie')


In [5]:
import psycopg2

def connect_to_postgres(dbname, user, password, host="localhost", port="5432"):
    try:
        # Connect to the PostgreSQL server
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        
        # Create a new cursor
        cursor = conn.cursor()
        
        # Print PostgreSQL version
        cursor.execute("SELECT version();")
        db_version = cursor.fetchone()
        print(f"Connected to: {db_version[0]}")
        
        # Close the cursor and connection
        cursor.close()
        conn.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
        return None

# Call the function
connect_to_postgres("gea_ikv_test", "bojan", "bojan123", "localhost", "5432")


Connected to: PostgreSQL 15.1, compiled by Visual C++ build 1914, 64-bit
